In [7]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

y_test = y_test - 769
y_train_valid = y_train_valid - 769
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

# For subject 1
X_train_valid_1 = X_train_valid[np.where(person_train_valid==0)[0]]
y_train_valid_1 = y_train_valid[np.where(person_train_valid==0)[0]]
X_test_1 = X_test[np.where(person_test==0)[0]]
y_test_1 = y_test[np.where(person_test==0)[0]]

class Dataset(torch.utils.data.Dataset):
    def __init__(self,X,Y):
        self.X = torch.FloatTensor(X)
        self.Y = torch.LongTensor(Y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index],self.Y[index]
    
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid,
                                                              test_size=0.2,shuffle=True)

train_set = Dataset(X_train,y_train)
val_set = Dataset(X_valid,y_valid)
test_set = Dataset(X_test, y_test)
print(X_train.shape)
train_loader = torch.utils.data.DataLoader(train_set,batch_size=60,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set,batch_size=20,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=40,shuffle=True)

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)
(1692, 22, 1000)


In [8]:
#目前这个最好
#只有FC有dropout
# [conv-relu]*2 -> 2*2 max-pooling -> [conv-relu]*3 -> 2*2 max_pooling -> (affine-relu)*2 -> affine -> softmax
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()  # initial the model
        self.conv1 = nn.Conv1d(22,40,kernel_size = 2,stride = 2) 
        self.bn1 = nn.BatchNorm1d(40)
        self.conv2 = nn.Conv1d(40,60,kernel_size = 3,stride = 1) 
        self.bn2 = nn.BatchNorm1d(60) 
        self.pool1 = nn.MaxPool1d(2,2) 
        
        self.conv3 = nn.Conv1d(60,80,kernel_size = 3, stride = 1) 
        self.bn3 = nn.BatchNorm1d(80)
        self.conv4 = nn.Conv1d(80,100,kernel_size = 3, stride = 1) 
        self.bn4 = nn.BatchNorm1d(100)
        self.conv5 = nn.Conv1d(100,120,kernel_size = 3, stride = 2) #120*122
        self.bn5 = nn.BatchNorm1d(120)
        self.pool2 = nn.MaxPool1d(2,2) #120*61
        
        self.fc1 = nn.Linear(120*61, 300) # input dim , output dim
        self.bn6 = nn.BatchNorm1d(300)
        self.drop1 = nn.Dropout(0.8)
        self.fc2 = nn.Linear(300,40)  
        self.bn7 = nn.BatchNorm1d(40)
        self.drop2 = nn.Dropout(0.8)
        self.fc3 = nn.Linear(40,4)
 
    def forward(self,x):
        x = self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x)))))))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(F.relu(self.bn5(self.conv5(x))))
        x = x.view(-1,120*61)
        
        x = self.drop1(F.relu(self.bn6(self.fc1(x))))
        x = self.drop2(F.relu(self.bn7(self.fc2(x))))
        x = self.fc3(x)
        return x
net = Net()
print(net)
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(net.parameters(),lr = 0.01)
optimizer = torch.optim.RMSprop(net.parameters(),lr = 0.001)

Net(
  (conv1): Conv1d(22, 40, kernel_size=(2,), stride=(2,))
  (bn1): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(40, 60, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(60, 80, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv1d(80, 100, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv1d(100, 120, kernel_size=(3,), stride=(2,))
  (bn5): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=7320, out_features=300, bias=True)
  (bn6): BatchNorm1d(300,

In [ ]:
for i in range(35,55):
    val_acc_history = []
    train_acc_history = []
    print('epoch number is ',i,'  ----------------------------')
    for epoch in range(i):
        for i , data in enumerate(train_loader, 0):
            inputs, labels = data

            net.train()

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _,predicted = torch.max(outputs.data, 1)
            train_acc = (predicted == labels).sum().item() / len(labels)

            if i % 2 == 1:
                net.eval()
                val_correct, val_total = 0, 0 
                for val_data in val_loader:
                    val_images, val_labels = val_data
                    val_outputs = net(val_images)
                    _, val_predicted = torch.max(val_outputs.data, 1)
                    val_total += val_labels.size(0)
                    val_correct += (val_predicted == val_labels).sum().item()
                val_acc = val_correct / val_total
                print('Epoch {} | Iter{} | Loss{:.4f} | TrainAcc{:.4f} | val acc {:.4f}'.format(
                    epoch, i , loss, train_acc, val_acc))
                #writer.add_scalar('Train/Loss',loss,epoch*len(trainloader) + i)
                #writer.add_scalar('Train/ACC',train_acc,epoch*len(trainloader) + i)
                #writer.add_scalar('VAL/ACC',val_acc,epoch*len(trainloader) + i)
        net.eval()
        train_correct, train_total = 0, 0
        for train_data in train_loader:
            train_inputs, train_labels = train_data
            train_outputs = net(train_inputs)
            _, train_predicted = torch.max(train_outputs.data, 1)
            train_total += train_labels.size(0)
            train_correct += (train_predicted == train_labels).sum().item()
        train_acc = train_correct / train_total
        train_acc_history.append(train_acc)
        val_correct, val_total = 0, 0
        for val_data in val_loader:
            val_images, val_labels = val_data
            val_outputs = net(val_images)
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()
        val_acc = val_correct / val_total
        val_acc_history.append(val_acc)
        #if val_acc == max(val_acc_history):
        #    net_best = Net()
        #    net_best.load_state_dict(net.state_dict())

    test_correct, test_total = 0, 0
    for test_data in test_loader:
        test_images, test_labels = test_data
        test_outputs = net(test_images)
        #test_outputs = net_best(test_images)
        _, test_predicted = torch.max(test_outputs.data, 1)
        test_total += test_labels.size(0)
        test_correct += (test_predicted == test_labels).sum().item()
    test_acc = test_correct / test_total
    print('Test accuracy is: ',test_acc)
    plt.plot(train_acc_history,)
    plt.plot(val_acc_history)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Model Accuracy')
    plt.legend(['train_acc','val_acc'])

epoch number is  35   ----------------------------
Epoch 0 | Iter1 | Loss1.5281 | TrainAcc0.2500 | val acc 0.2411
Epoch 0 | Iter3 | Loss1.5921 | TrainAcc0.2333 | val acc 0.2837
Epoch 0 | Iter5 | Loss1.7820 | TrainAcc0.2000 | val acc 0.2766
Epoch 0 | Iter7 | Loss1.7163 | TrainAcc0.2167 | val acc 0.2553
Epoch 0 | Iter9 | Loss1.5161 | TrainAcc0.2833 | val acc 0.2884
Epoch 0 | Iter11 | Loss1.7454 | TrainAcc0.2833 | val acc 0.2695
Epoch 0 | Iter13 | Loss1.6019 | TrainAcc0.2500 | val acc 0.2813
Epoch 0 | Iter15 | Loss1.5916 | TrainAcc0.2833 | val acc 0.2530
Epoch 0 | Iter17 | Loss1.5148 | TrainAcc0.3167 | val acc 0.2742
Epoch 0 | Iter19 | Loss1.5629 | TrainAcc0.2667 | val acc 0.2530
Epoch 0 | Iter21 | Loss1.5274 | TrainAcc0.3167 | val acc 0.2813
Epoch 0 | Iter23 | Loss1.6797 | TrainAcc0.1333 | val acc 0.2530
Epoch 0 | Iter25 | Loss1.5209 | TrainAcc0.2500 | val acc 0.2695
Epoch 0 | Iter27 | Loss1.5185 | TrainAcc0.2500 | val acc 0.2388
Epoch 1 | Iter1 | Loss1.5514 | TrainAcc0.1833 | val acc 0.

Epoch 9 | Iter5 | Loss1.2604 | TrainAcc0.3833 | val acc 0.4350
Epoch 9 | Iter7 | Loss1.2389 | TrainAcc0.4333 | val acc 0.4492
Epoch 9 | Iter9 | Loss1.1864 | TrainAcc0.4833 | val acc 0.4539
Epoch 9 | Iter11 | Loss1.2291 | TrainAcc0.5167 | val acc 0.4090
Epoch 9 | Iter13 | Loss1.1435 | TrainAcc0.4333 | val acc 0.4421
Epoch 9 | Iter15 | Loss1.3098 | TrainAcc0.4167 | val acc 0.4421
Epoch 9 | Iter17 | Loss1.2302 | TrainAcc0.4833 | val acc 0.3759
Epoch 9 | Iter19 | Loss1.0884 | TrainAcc0.5000 | val acc 0.4043
Epoch 9 | Iter21 | Loss1.2507 | TrainAcc0.4333 | val acc 0.4421
Epoch 9 | Iter23 | Loss1.1826 | TrainAcc0.4333 | val acc 0.4563
Epoch 9 | Iter25 | Loss1.1655 | TrainAcc0.4833 | val acc 0.4184
Epoch 9 | Iter27 | Loss1.2046 | TrainAcc0.3667 | val acc 0.4113
Epoch 10 | Iter1 | Loss1.1466 | TrainAcc0.4500 | val acc 0.4350
Epoch 10 | Iter3 | Loss1.1746 | TrainAcc0.4667 | val acc 0.4326
Epoch 10 | Iter5 | Loss1.3163 | TrainAcc0.3500 | val acc 0.4657
Epoch 10 | Iter7 | Loss1.1973 | TrainAcc0.4

Epoch 18 | Iter7 | Loss1.0291 | TrainAcc0.5000 | val acc 0.4303
Epoch 18 | Iter9 | Loss0.9319 | TrainAcc0.5667 | val acc 0.4657
Epoch 18 | Iter11 | Loss0.7809 | TrainAcc0.7000 | val acc 0.4586
Epoch 18 | Iter13 | Loss0.9493 | TrainAcc0.6000 | val acc 0.4610
Epoch 18 | Iter15 | Loss0.8624 | TrainAcc0.5667 | val acc 0.4444
Epoch 18 | Iter17 | Loss0.9140 | TrainAcc0.5333 | val acc 0.4326
Epoch 18 | Iter19 | Loss0.9948 | TrainAcc0.5667 | val acc 0.4444
Epoch 18 | Iter21 | Loss0.8298 | TrainAcc0.6667 | val acc 0.4492
Epoch 18 | Iter23 | Loss0.7315 | TrainAcc0.6500 | val acc 0.4775
Epoch 18 | Iter25 | Loss0.9215 | TrainAcc0.6000 | val acc 0.4184
Epoch 18 | Iter27 | Loss0.7696 | TrainAcc0.6667 | val acc 0.4350
Epoch 19 | Iter1 | Loss0.9932 | TrainAcc0.5833 | val acc 0.4350
Epoch 19 | Iter3 | Loss0.8227 | TrainAcc0.6500 | val acc 0.4610
Epoch 19 | Iter5 | Loss0.9312 | TrainAcc0.6000 | val acc 0.4634
Epoch 19 | Iter7 | Loss0.7921 | TrainAcc0.6833 | val acc 0.4492
Epoch 19 | Iter9 | Loss0.7226 |

Epoch 27 | Iter9 | Loss0.6491 | TrainAcc0.7000 | val acc 0.4161
Epoch 27 | Iter11 | Loss0.5593 | TrainAcc0.7667 | val acc 0.4397
Epoch 27 | Iter13 | Loss0.6598 | TrainAcc0.7000 | val acc 0.4610
Epoch 27 | Iter15 | Loss0.5806 | TrainAcc0.8500 | val acc 0.4563
Epoch 27 | Iter17 | Loss0.8119 | TrainAcc0.6667 | val acc 0.4303
Epoch 27 | Iter19 | Loss0.5543 | TrainAcc0.7500 | val acc 0.4444
Epoch 27 | Iter21 | Loss0.6904 | TrainAcc0.7000 | val acc 0.4374
Epoch 27 | Iter23 | Loss0.5810 | TrainAcc0.7833 | val acc 0.4468
Epoch 27 | Iter25 | Loss0.5854 | TrainAcc0.7167 | val acc 0.4303
Epoch 27 | Iter27 | Loss0.7460 | TrainAcc0.7000 | val acc 0.4397
Epoch 28 | Iter1 | Loss0.6982 | TrainAcc0.6833 | val acc 0.4634
Epoch 28 | Iter3 | Loss0.4882 | TrainAcc0.7833 | val acc 0.4539
Epoch 28 | Iter5 | Loss0.5867 | TrainAcc0.7167 | val acc 0.4515
Epoch 28 | Iter7 | Loss0.5821 | TrainAcc0.8333 | val acc 0.4515
Epoch 28 | Iter9 | Loss0.5224 | TrainAcc0.7167 | val acc 0.4444
Epoch 28 | Iter11 | Loss0.6825 

Epoch 1 | Iter9 | Loss0.5272 | TrainAcc0.7833 | val acc 0.4586
Epoch 1 | Iter11 | Loss0.4597 | TrainAcc0.8000 | val acc 0.4374
Epoch 1 | Iter13 | Loss0.5200 | TrainAcc0.7333 | val acc 0.4326
Epoch 1 | Iter15 | Loss0.5207 | TrainAcc0.8000 | val acc 0.4563
Epoch 1 | Iter17 | Loss0.4263 | TrainAcc0.9000 | val acc 0.4634
Epoch 1 | Iter19 | Loss0.4950 | TrainAcc0.7500 | val acc 0.4492
Epoch 1 | Iter21 | Loss0.3842 | TrainAcc0.8167 | val acc 0.4255
Epoch 1 | Iter23 | Loss0.5328 | TrainAcc0.7500 | val acc 0.4539
Epoch 1 | Iter25 | Loss0.4370 | TrainAcc0.8667 | val acc 0.4634
Epoch 1 | Iter27 | Loss0.5227 | TrainAcc0.8333 | val acc 0.4515
Epoch 2 | Iter1 | Loss0.4213 | TrainAcc0.8333 | val acc 0.4634
Epoch 2 | Iter3 | Loss0.4207 | TrainAcc0.8500 | val acc 0.4681
Epoch 2 | Iter5 | Loss0.3503 | TrainAcc0.8833 | val acc 0.4350
Epoch 2 | Iter7 | Loss0.3854 | TrainAcc0.9000 | val acc 0.4610
Epoch 2 | Iter9 | Loss0.4746 | TrainAcc0.8333 | val acc 0.4775
Epoch 2 | Iter11 | Loss0.4454 | TrainAcc0.8500

...

Epoch 47 | Iter5 | Loss0.0900 | TrainAcc0.9500 | val acc 0.4823
Epoch 47 | Iter7 | Loss0.1584 | TrainAcc0.9333 | val acc 0.4917
Epoch 47 | Iter9 | Loss0.1363 | TrainAcc0.9333 | val acc 0.4941
Epoch 47 | Iter11 | Loss1.3533 | TrainAcc0.9167 | val acc 0.4775
Epoch 47 | Iter13 | Loss0.1801 | TrainAcc0.9333 | val acc 0.4799
Epoch 47 | Iter15 | Loss0.5278 | TrainAcc0.9333 | val acc 0.4823
Epoch 47 | Iter17 | Loss0.1207 | TrainAcc0.9667 | val acc 0.4870
Epoch 47 | Iter19 | Loss0.1716 | TrainAcc0.9500 | val acc 0.4894
Epoch 47 | Iter21 | Loss0.0933 | TrainAcc0.9833 | val acc 0.5059
Epoch 47 | Iter23 | Loss0.0711 | TrainAcc0.9667 | val acc 0.5059
Epoch 47 | Iter25 | Loss0.1915 | TrainAcc0.9333 | val acc 0.5177
Epoch 47 | Iter27 | Loss0.1183 | TrainAcc0.9500 | val acc 0.5225
Epoch 48 | Iter1 | Loss0.0561 | TrainAcc0.9833 | val acc 0.5106
Epoch 48 | Iter3 | Loss0.1981 | TrainAcc0.9500 | val acc 0.5272
Epoch 48 | Iter5 | Loss0.0454 | TrainAcc0.9833 | val acc 0.5106
Epoch 48 | Iter7 | Loss0.0758 |

Epoch 5 | Iter7 | Loss0.0615 | TrainAcc0.9833 | val acc 0.4988
Epoch 5 | Iter9 | Loss0.1467 | TrainAcc0.9500 | val acc 0.4941
Epoch 5 | Iter11 | Loss0.5614 | TrainAcc0.9333 | val acc 0.4941
Epoch 5 | Iter13 | Loss0.1259 | TrainAcc0.9500 | val acc 0.4846
Epoch 5 | Iter15 | Loss0.0529 | TrainAcc0.9833 | val acc 0.4775
Epoch 5 | Iter17 | Loss0.0847 | TrainAcc0.9667 | val acc 0.4870
Epoch 5 | Iter19 | Loss0.0537 | TrainAcc0.9833 | val acc 0.4870
Epoch 5 | Iter21 | Loss0.0717 | TrainAcc0.9667 | val acc 0.4823
Epoch 5 | Iter23 | Loss0.0479 | TrainAcc0.9833 | val acc 0.4846
Epoch 5 | Iter25 | Loss0.0728 | TrainAcc0.9500 | val acc 0.4799
Epoch 5 | Iter27 | Loss0.0289 | TrainAcc1.0000 | val acc 0.4823
Epoch 6 | Iter1 | Loss0.1225 | TrainAcc0.9667 | val acc 0.4846
Epoch 6 | Iter3 | Loss0.1083 | TrainAcc0.9667 | val acc 0.4846
Epoch 6 | Iter5 | Loss0.0969 | TrainAcc0.9667 | val acc 0.4870
Epoch 6 | Iter7 | Loss0.2705 | TrainAcc0.9167 | val acc 0.4965
Epoch 6 | Iter9 | Loss0.0179 | TrainAcc1.0000 

Epoch 14 | Iter11 | Loss0.0491 | TrainAcc1.0000 | val acc 0.4823
Epoch 14 | Iter13 | Loss0.3303 | TrainAcc0.9167 | val acc 0.4941
Epoch 14 | Iter15 | Loss0.0238 | TrainAcc1.0000 | val acc 0.4846
Epoch 14 | Iter17 | Loss0.0797 | TrainAcc0.9667 | val acc 0.4965
Epoch 14 | Iter19 | Loss0.0533 | TrainAcc1.0000 | val acc 0.4823
Epoch 14 | Iter21 | Loss0.1749 | TrainAcc0.9000 | val acc 0.4917
Epoch 14 | Iter23 | Loss0.0605 | TrainAcc0.9667 | val acc 0.4965
Epoch 14 | Iter25 | Loss0.1965 | TrainAcc0.9500 | val acc 0.5035
Epoch 14 | Iter27 | Loss0.1406 | TrainAcc0.9333 | val acc 0.5083
Epoch 15 | Iter1 | Loss0.1543 | TrainAcc0.9333 | val acc 0.5177
Epoch 15 | Iter3 | Loss0.1733 | TrainAcc0.9500 | val acc 0.5130
Epoch 15 | Iter5 | Loss0.0953 | TrainAcc0.9500 | val acc 0.4775
Epoch 15 | Iter7 | Loss0.6091 | TrainAcc0.9667 | val acc 0.4965
Epoch 15 | Iter9 | Loss0.1412 | TrainAcc0.9500 | val acc 0.4965
Epoch 15 | Iter11 | Loss0.0602 | TrainAcc0.9833 | val acc 0.5012
Epoch 15 | Iter13 | Loss0.0928

Epoch 23 | Iter13 | Loss0.0698 | TrainAcc0.9500 | val acc 0.4941
Epoch 23 | Iter15 | Loss0.0627 | TrainAcc0.9667 | val acc 0.4941
Epoch 23 | Iter17 | Loss0.0626 | TrainAcc0.9833 | val acc 0.5012
Epoch 23 | Iter19 | Loss0.0781 | TrainAcc0.9833 | val acc 0.4988
Epoch 23 | Iter21 | Loss0.1004 | TrainAcc0.9500 | val acc 0.4965
Epoch 23 | Iter23 | Loss0.0877 | TrainAcc0.9667 | val acc 0.5012
Epoch 23 | Iter25 | Loss0.3718 | TrainAcc0.9333 | val acc 0.4941
Epoch 23 | Iter27 | Loss0.0783 | TrainAcc0.9667 | val acc 0.4917
Epoch 24 | Iter1 | Loss0.1055 | TrainAcc0.9500 | val acc 0.4823
Epoch 24 | Iter3 | Loss0.1459 | TrainAcc0.9500 | val acc 0.4846
Epoch 24 | Iter5 | Loss0.1542 | TrainAcc0.9167 | val acc 0.4846
Epoch 24 | Iter7 | Loss0.1423 | TrainAcc0.9333 | val acc 0.4870
Epoch 24 | Iter9 | Loss0.0450 | TrainAcc0.9833 | val acc 0.4846
Epoch 24 | Iter11 | Loss0.1679 | TrainAcc0.9333 | val acc 0.4894
Epoch 24 | Iter13 | Loss0.1166 | TrainAcc0.9667 | val acc 0.4799
Epoch 24 | Iter15 | Loss0.0999

Epoch 32 | Iter15 | Loss0.0703 | TrainAcc0.9667 | val acc 0.4657
Epoch 32 | Iter17 | Loss0.1056 | TrainAcc0.9667 | val acc 0.4823
Epoch 32 | Iter19 | Loss0.0771 | TrainAcc0.9500 | val acc 0.4846
Epoch 32 | Iter21 | Loss0.1850 | TrainAcc0.9333 | val acc 0.4917
Epoch 32 | Iter23 | Loss0.0485 | TrainAcc0.9833 | val acc 0.4917
Epoch 32 | Iter25 | Loss0.1430 | TrainAcc0.9500 | val acc 0.4870
Epoch 32 | Iter27 | Loss0.1238 | TrainAcc0.9667 | val acc 0.4704
Epoch 33 | Iter1 | Loss0.1174 | TrainAcc0.9333 | val acc 0.4681
Epoch 33 | Iter3 | Loss0.0908 | TrainAcc0.9833 | val acc 0.4775
Epoch 33 | Iter5 | Loss0.1004 | TrainAcc0.9667 | val acc 0.4894
Epoch 33 | Iter7 | Loss0.1867 | TrainAcc0.9167 | val acc 0.4870
Epoch 33 | Iter9 | Loss0.1244 | TrainAcc0.9500 | val acc 0.4965
Epoch 33 | Iter11 | Loss0.1861 | TrainAcc0.9333 | val acc 0.4988
Epoch 33 | Iter13 | Loss0.1674 | TrainAcc0.9667 | val acc 0.4965
Epoch 33 | Iter15 | Loss0.0646 | TrainAcc0.9833 | val acc 0.5083
Epoch 33 | Iter17 | Loss0.1695